In [3]:
#initializing pyspark
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'/Users/kpathak/opt/anaconda3/envs/mamlTesting/lib/python3.8/site-packages/pyspark'

In [4]:
#https://naomi-fridman.medium.com/install-pyspark-to-run-on-jupyter-notebook-on-windows-4ec2009de21f
#https://github.com/spark-examples/pyspark-examples/
#https://www.datacamp.com/tutorial/pyspark-tutorial-getting-started-with-pyspark
#https://www.guru99.com/pyspark-tutorial.html
#https://sparkbyexamples.com/pyspark/install-pyspark-in-anaconda-jupyter-notebook/
#follow conda install for pyspark and findaprk

In [5]:
from pyspark.sql import SparkSession
sc =  SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
spark= sc
print(spark)

22/10/25 19:01:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/25 19:01:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
spark

In [133]:
# spark.stop()

In [134]:
#create spark data frame
import pandas as pd    
data = [['Scott', 50], ['Jeff', 45], ['Thomas', 54],['Ann',34]] 
  
# Create the pandas DataFrame 
pandasDF = pd.DataFrame(data, columns = ['Name', 'Age']) 
  
# print dataframe. 
print(pandasDF)
sparkDF=spark.createDataFrame(pandasDF) 
sparkDF.printSchema()
sparkDF.show()

     Name  Age
0   Scott   50
1    Jeff   45
2  Thomas   54
3     Ann   34
root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)

+------+---+
|  Name|Age|
+------+---+
| Scott| 50|
|  Jeff| 45|
|Thomas| 54|
|   Ann| 34|
+------+---+



In [135]:
#paralleziation
nums= spark.sparkContext.parallelize([1,2,3,4])

In [138]:
nums.take(1)

[1]

In [140]:
%%time
squared = nums.map(lambda x: x*x).collect()
for num in squared:
    print('%i ' % (num))

1 
4 
9 
16 
CPU times: user 4.17 ms, sys: 3.82 ms, total: 7.99 ms
Wall time: 75.2 ms


In [141]:
%%time
num1=[1,2,3,4]
for i in num1:
    print('%i '% (i*i))

1 
4 
9 
16 
CPU times: user 402 µs, sys: 290 µs, total: 692 µs
Wall time: 473 µs


In [10]:
#SQLContext allows connecting the engine with different data sources. It is used to initiate the functionalities of Spark SQL.
from pyspark.sql import Row
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [11]:
#create tuple
tuple=[('John',19),('Smith',29),('Adam',35),('Henry',50)]

In [12]:
#creating rdd
rdd = spark.sparkContext.parallelize(tuple)

In [13]:
#converting tuples
rdd.map(lambda x: Row(name=x[0], age=int(x[1]))).collect()

[Row(name='John', age=19),
 Row(name='Smith', age=29),
 Row(name='Adam', age=35),
 Row(name='Henry', age=50)]

In [14]:
#create dataframe
ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
DF_ppl = sqlContext.createDataFrame(ppl)
DF_ppl.show()

+-----+---+
| name|age|
+-----+---+
| John| 19|
|Smith| 29|
| Adam| 35|
|Henry| 50|
+-----+---+



In [15]:
#print schema
DF_ppl.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [16]:
#Machine learning example
url = "https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv"
spark.sparkContext.addFile(url)
from pyspark import SparkFiles
sqlContext = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)

22/10/25 19:03:30 WARN SparkContext: The path https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv has been added already. Overwriting of added paths is not supported in the current version.


In [17]:
#data fetchung from uri
df = sqlContext.read.csv(SparkFiles.get("adult_data.csv"), header=True, inferSchema= True)

In [18]:
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [19]:
df.count()

48842

In [20]:
df.show(5, truncate = False)

+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|x  |age|workclass|fnlwgt|education   |educational-num|marital-status    |occupation       |relationship|race |gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|1  |25 |Private  |226802|11th        |7              |Never-married     |Machine-op-inspct|Own-child   |Black|Male  |0           |0           |40            |United-States |<=50K |
|2  |38 |Private  |89814 |HS-grad     |9              |Married-civ-spouse|Farming-fishing  |Husband     |White|Male  |0           |0           |50            |United-States |<=50K |
|3  |28 |Local-gov|336951|Assoc-acdm  |12             |Married-civ-spouse|Protective-serv 

In [21]:
df = sqlContext.read.csv(SparkFiles.get("adult_data.csv"), header=True, inferSchema= False)
df.printSchema()

root
 |-- x: string (nullable = true)
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: string (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: string (nullable = true)
 |-- capital-loss: string (nullable = true)
 |-- hours-per-week: string (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [46]:
# Import all from `sql.types`
from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 
# List of continuous features
CONTI_FEATURES  = ['age', 'fnlwgt','capital-gain', 'educational-num', 'capital-loss', 'hours-per-week']
# Convert the type
df_string = convertColumn(df, CONTI_FEATURES, FloatType())
df_string.printSchema()

root
 |-- x: string (nullable = true)
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: float (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: float (nullable = true)
 |-- capital-loss: float (nullable = true)
 |-- hours-per-week: float (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [23]:
#select column
df.select(df.age,df.fnlwgt).show(5)
# df.select(df["age"],df["fnlwgt"]).show()


+---+------+
|age|fnlwgt|
+---+------+
| 25|226802|
| 38| 89814|
| 28|336951|
| 44|160323|
| 18|103497|
+---+------+
only showing top 5 rows



In [24]:
#Groupby
df2=df.groupBy(df['education']).count()
df2.show()

+------------+-----+
|   education|count|
+------------+-----+
|        10th| 1389|
|     Masters| 2657|
|     5th-6th|  509|
|  Assoc-acdm| 1601|
|   Assoc-voc| 2061|
|     7th-8th|  955|
|         9th|  756|
|     HS-grad|15784|
|   Bachelors| 8025|
|        11th| 1812|
|     1st-4th|  247|
|   Preschool|   83|
|        12th|  657|
|   Doctorate|  594|
|Some-college|10878|
| Prof-school|  834|
+------------+-----+



In [242]:
df1=df.groupBy(df['education']).count()
df1=df1.sort(df1['count'],ascending=False)
df1.show()

+------------+-----+
|   education|count|
+------------+-----+
|     HS-grad|15784|
|Some-college|10878|
|   Bachelors| 8025|
|     Masters| 2657|
|   Assoc-voc| 2061|
|        11th| 1812|
|  Assoc-acdm| 1601|
|        10th| 1389|
|     7th-8th|  955|
| Prof-school|  834|
|         9th|  756|
|        12th|  657|
|   Doctorate|  594|
|     5th-6th|  509|
|     1st-4th|  247|
|   Preschool|   83|
+------------+-----+



In [243]:
#describe data
df.describe().show()

+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+------------------+------------------+--------------+------+
|summary|                 x|               age|  workclass|            fnlwgt|   education|   educational-num|marital-status|      occupation|relationship|              race|gender|      capital-gain|      capital-loss|    hours-per-week|native-country|income|
+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+------------------+------------------+--------------+------+
|  count|             48842|             48842|      48842|             48842|       48842|             48842|         48842|           48842|       48842|             48842| 48842|             48842|             4884

In [248]:
#describe a particular column
df.describe('capital-gain').show()

+-------+------------------+
|summary|      capital-gain|
+-------+------------------+
|  count|             48842|
|   mean|1079.0676262233324|
| stddev| 7452.019057655406|
|    min|                 0|
|    max|             99999|
+-------+------------------+



In [251]:
#creating a cross tab
df1=df.crosstab('age', 'workclass')
df1.show()

+-------------+---+-----------+---------+------------+-------+------------+----------------+---------+-----------+
|age_workclass|  ?|Federal-gov|Local-gov|Never-worked|Private|Self-emp-inc|Self-emp-not-inc|State-gov|Without-pay|
+-------------+---+-----------+---------+------------+-------+------------+----------------+---------+-----------+
|           69| 32|          2|        9|           0|     66|           7|              26|        7|          0|
|           88|  0|          0|        0|           0|      2|           0|               4|        0|          0|
|           56| 17|         23|       38|           0|    351|          30|              81|       24|          0|
|           42| 22|         46|       98|           0|    755|          52|             128|       64|          0|
|           24| 56|         21|       56|           0|    986|          10|              35|       42|          0|
|           37| 18|         51|       91|           0|    899|          47|     

In [252]:
#drop columns
df.drop('education_num').columns

['x',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'educational-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'income']

In [253]:
#filter data 
df.filter(df.age > 40).count()

20211

In [258]:
#grouby statistics
df.groupby(df['marital-status']).agg({'capital-gain' : 'mean'}).show()

+--------------------+------------------+
|      marital-status| avg(capital-gain)|
+--------------------+------------------+
|           Separated| 581.8424836601307|
|       Never-married|  384.382639449029|
|Married-spouse-ab...| 629.0047770700637|
|            Divorced| 793.6755615860094|
|             Widowed| 603.6442687747035|
|   Married-AF-spouse|2971.6216216216217|
|  Married-civ-spouse|1739.7006121810625|
+--------------------+------------------+



In [259]:
#Data preprocessing

In [281]:
from pyspark.sql.functions import *

# 1 Select the column
age_square = df.select(df["age"]*df["age"])

# 2 Apply the transformation and add it to the DataFrame
df = df.withColumn("age_square", df["age"]*df["age"])
df.show(5)

+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+----------+
|  x|age|workclass|fnlwgt|   education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|age_square|
+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+----------+
|  1| 25|  Private|226802|        11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|       625|
|  2| 38|  Private| 89814|     HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|            50| United-States| <=50K|      1444|
|  3| 28|Local-gov|336951|  Assoc-a

In [292]:
#select columns
df1 = df.select(df['age'], df['age_square'], df['workclass'], df['fnlwgt'], df['education'], df['educational-num'], df['marital-status'],
           df['occupation'], df['relationship'], df['race'], df['gender'], df['capital-gain'], df['capital-loss'],df['hours-per-week'], df['native-country'])
df1.show(5)

+---+----------+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+
|age|age_square|workclass|fnlwgt|   education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|
+---+----------+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+
| 25|       625|  Private|226802|        11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States|
| 38|      1444|  Private| 89814|     HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|            50| United-States|
| 28|       784|Local-gov|336951|  Assoc-acdm|             12|Married-civ-spouse|  Protect

In [293]:
df1.count()

48842

In [299]:
#filter
print(df.filter(df['native-country'] == 'Holand-Netherlands').count())
df.groupby(df['native-country']).agg({'native-country': 'count'}).show(5)

1
+--------------+---------------------+
|native-country|count(native-country)|
+--------------+---------------------+
|   Philippines|                  295|
|       Germany|                  206|
|      Cambodia|                   28|
|        France|                   38|
|        Greece|                   49|
+--------------+---------------------+
only showing top 5 rows



In [48]:
#removing something
df_remove = df.filter(df['native-country'] != 'Holand-Netherlands')
df_remove.count()

48841

In [319]:
#looking at group states for workclass
df.groupBy(df['workclass']).count().collect()

[Row(workclass='Self-emp-not-inc', count=3862),
 Row(workclass='Local-gov', count=3136),
 Row(workclass='State-gov', count=1981),
 Row(workclass='Private', count=33906),
 Row(workclass='Without-pay', count=21),
 Row(workclass='Federal-gov', count=1432),
 Row(workclass='Never-worked', count=10),
 Row(workclass='?', count=2799),
 Row(workclass='Self-emp-inc', count=1695)]

In [32]:
#building data processing pipeline
### Example encoder
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

string_Indexer = StringIndexer(inputCol="workclass", outputCol="workclass_encoded")
model = string_Indexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(dropLast=False, inputCol="workclass_encoded", outputCol="workclass_vec").fit(indexed)
encoded = encoder.transform(indexed)
encoded.show(5)

+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+-------------+
|  x|age|workclass|fnlwgt|   education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|workclass_encoded|workclass_vec|
+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+-------------+
|  1| 25|  Private|226802|        11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|              0.0|(9,[0],[1.0])|
|  2| 38|  Private| 89814|     HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|

In [38]:
encoded.select('workclass_encoded','workclass_vec').show()

+-----------------+-------------+
|workclass_encoded|workclass_vec|
+-----------------+-------------+
|              0.0|(9,[0],[1.0])|
|              0.0|(9,[0],[1.0])|
|              2.0|(9,[2],[1.0])|
|              0.0|(9,[0],[1.0])|
|              3.0|(9,[3],[1.0])|
|              0.0|(9,[0],[1.0])|
|              3.0|(9,[3],[1.0])|
|              1.0|(9,[1],[1.0])|
|              0.0|(9,[0],[1.0])|
|              0.0|(9,[0],[1.0])|
|              0.0|(9,[0],[1.0])|
|              6.0|(9,[6],[1.0])|
|              0.0|(9,[0],[1.0])|
|              3.0|(9,[3],[1.0])|
|              0.0|(9,[0],[1.0])|
|              0.0|(9,[0],[1.0])|
|              4.0|(9,[4],[1.0])|
|              0.0|(9,[0],[1.0])|
|              0.0|(9,[0],[1.0])|
|              0.0|(9,[0],[1.0])|
+-----------------+-------------+
only showing top 20 rows



In [42]:
#create data-testing
#create the structure of schema
# schema = StructType().add("id","integer").add("name","string").add("qualification","string").add("age", "integer").add("gender", "string")
# data = [
#     (1,'John',"B.A.", 20, "Male"),
#     (2,'Martha',"B.Com.", 20, "Female"),
#     (3,'Mona',"B.Com.", 21, "Female"),
#     (4,'Harish',"B.Sc.", 22, "Male"),
#     (5,'Jonny',"B.A.", 22, "Male"),
#     (6,'Maria',"B.A.", 23, "Female"),
#     (7,'Monalisa',"B.A.", 21, "Female")
# ]
# df_test = spark.createDataFrame(data, schema=schema)
# qualification_indexer = StringIndexer(inputCol="qualification", outputCol="qualificationIndex")
# #Fits a model to the input dataset with optional parameters.
# df1_test = qualification_indexer.fit(df_test).transform(df_test)
# df1_test.show()

In [41]:
#onehotencoder to qualificationIndex
# onehotencoder_qualification_vector = OneHotEncoder(inputCol="qualificationIndex", outputCol="qualification_vec")
# df11_test = onehotencoder_qualification_vector.fit(df1_test).transform(df1_test)
# df11_test.show()

In [94]:
#Build the pipeline
from pyspark.ml import Pipeline
CATE_FEATURES = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country','income']
# CATE_FEATURES = ['workclass', 'education']

stages = [] # stages in our Pipeline
for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
# Convert label into label indices using the StringIndexer
label_stringIdx =  StringIndexer(inputCol="income", outputCol="newlabel")
stages += [label_stringIdx]

In [95]:
stages

[StringIndexer_d13bc29328d3,
 OneHotEncoder_5d817771b9e2,
 StringIndexer_834dbdee2656,
 OneHotEncoder_2c99eff7c4a2,
 StringIndexer_722880280649,
 OneHotEncoder_7fe1de319cae,
 StringIndexer_ffb3651de9b1,
 OneHotEncoder_5f88ad0f1f8e,
 StringIndexer_424efa9aacbb,
 OneHotEncoder_a65bde17d2de,
 StringIndexer_a95c9282cb16,
 OneHotEncoder_3eac99a70a3b,
 StringIndexer_4ea144e46c03,
 OneHotEncoder_510482c280f6,
 StringIndexer_8b346fd910c8,
 OneHotEncoder_4cd872fdede8,
 StringIndexer_6a30065ccbc9,
 OneHotEncoder_fc14e6f6b8f8,
 StringIndexer_d28b980e817e]

In [96]:
# assembling all features
CONTI_FEATURES  = ['age', 'fnlwgt','capital-gain', 'educational-num', 'capital-loss', 'hours-per-week']
# CONTI_FEATURES  = ['age', 'fnlwgt']

assemblerInputs = [c + "classVec" for c in CATE_FEATURES] + CONTI_FEATURES
assemblerInputs

['workclassclassVec',
 'educationclassVec',
 'marital-statusclassVec',
 'occupationclassVec',
 'relationshipclassVec',
 'raceclassVec',
 'genderclassVec',
 'native-countryclassVec',
 'incomeclassVec',
 'age',
 'fnlwgt',
 'capital-gain',
 'educational-num',
 'capital-loss',
 'hours-per-week']

In [97]:
#final vector assemblers
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]
stages

[StringIndexer_d13bc29328d3,
 OneHotEncoder_5d817771b9e2,
 StringIndexer_834dbdee2656,
 OneHotEncoder_2c99eff7c4a2,
 StringIndexer_722880280649,
 OneHotEncoder_7fe1de319cae,
 StringIndexer_ffb3651de9b1,
 OneHotEncoder_5f88ad0f1f8e,
 StringIndexer_424efa9aacbb,
 OneHotEncoder_a65bde17d2de,
 StringIndexer_a95c9282cb16,
 OneHotEncoder_3eac99a70a3b,
 StringIndexer_4ea144e46c03,
 OneHotEncoder_510482c280f6,
 StringIndexer_8b346fd910c8,
 OneHotEncoder_4cd872fdede8,
 StringIndexer_6a30065ccbc9,
 OneHotEncoder_fc14e6f6b8f8,
 StringIndexer_d28b980e817e,
 VectorAssembler_c3bdddd766a0]

In [98]:
df_remove.columns

['x',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'educational-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'income']

In [99]:
#data type conversion
# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 
# List of continuous features
CONTI_FEATURES  = ['age', 'fnlwgt','capital-gain', 'educational-num', 'capital-loss', 'hours-per-week']
# Convert the type
df_remove = convertColumn(df_remove, CONTI_FEATURES, FloatType())
df_remove.printSchema()

root
 |-- x: string (nullable = true)
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: float (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: float (nullable = true)
 |-- capital-loss: float (nullable = true)
 |-- hours-per-week: float (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [100]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df_remove)
model = pipelineModel.transform(df_remove)
model.take(1)

[Row(x='1', age=25.0, workclass='Private', fnlwgt=226802.0, education='11th', educational-num=7.0, marital-status='Never-married', occupation='Machine-op-inspct', relationship='Own-child', race='Black', gender='Male', capital-gain=0.0, capital-loss=0.0, hours-per-week=40.0, native-country='United-States', income='<=50K', workclassIndex=0.0, workclassclassVec=SparseVector(8, {0: 1.0}), educationIndex=5.0, educationclassVec=SparseVector(15, {5: 1.0}), marital-statusIndex=1.0, marital-statusclassVec=SparseVector(6, {1: 1.0}), occupationIndex=6.0, occupationclassVec=SparseVector(14, {6: 1.0}), relationshipIndex=2.0, relationshipclassVec=SparseVector(5, {2: 1.0}), raceIndex=1.0, raceclassVec=SparseVector(4, {1: 1.0}), genderIndex=0.0, genderclassVec=SparseVector(1, {0: 1.0}), native-countryIndex=0.0, native-countryclassVec=SparseVector(40, {0: 1.0}), incomeIndex=0.0, incomeclassVec=SparseVector(1, {0: 1.0}), newlabel=0.0, features=SparseVector(100, {0: 1.0, 13: 1.0, 24: 1.0, 35: 1.0, 45: 1.

In [101]:
#Build a logistic regression model
#To make the computation faster, you convert model to a DataFrame.
#You need to select newlabel and features from model using map.
from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x: (x["newlabel"], DenseVector(x["features"])))

In [102]:
df_train = sqlContext.createDataFrame(input_data, ["label", "features"])
df_train.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[1.0,0.0,0.0,0.0,...|
|  0.0|[1.0,0.0,0.0,0.0,...|
+-----+--------------------+
only showing top 2 rows



In [103]:
# Split the data into train and test sets
train_data, test_data = df_train.randomSplit([.8,.2],seed=1234)
train_data.count( ), test_data.count()

(39058, 9783)

In [104]:
train_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|       29699|
|  1.0|        9359|
+-----+------------+



In [107]:
test_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        7455|
|  1.0|        2328|
+-----+------------+



In [108]:
#building logistic regression
# Import `LinearRegression`
from pyspark.ml.classification import LogisticRegression

# Initialize `lr`
lr = LogisticRegression(labelCol="label",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.3)

# Fit the data to the model
linearModel = lr.fit(train_data)
linearModel

22/10/26 12:21:38 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/10/26 12:21:38 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


LogisticRegressionModel: uid=LogisticRegression_a204be12eb61, numClasses=2, numFeatures=100

In [109]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(linearModel.coefficients))
print("Intercept: " + str(linearModel.intercept))

Coefficients: [-0.03416816214840661,-0.03293224579727304,0.018893121473898048,-0.12810404490487237,-0.03907540103820323,0.19767975898978352,0.15470840740025515,-0.10987676402208173,-0.10732534641615692,-0.041495549249503084,0.16717957884953566,0.2605054615995495,0.0017138317111527821,-0.17127621732687565,0.0018362280631443533,-0.17277900340141406,-0.21275752582970248,0.39522140047992005,-0.19943706754962034,-0.1497797970933461,0.3864639895638318,-0.17630901438373314,-0.20772314477467202,0.29406712507551414,-0.2481983164988183,-0.12806572288472737,-0.13542560444300447,-0.12690008308238676,-0.10771812392366444,0.1706740177354208,-0.017695709882279805,0.23563510546295993,-0.08099206271364243,0.03835175594235224,-0.19715713548154026,-0.12263151780093609,-0.12852668543261234,-0.05057587255783086,-0.17622964934046156,-0.19299694715356758,0.08563316099996775,0.07256061119346253,-0.20779670494751068,0.24908853558249425,-0.14287365247963157,-0.22733400109807195,-0.16671089794767888,0.3030675333

In [110]:
# Make predictions on test data using the transform() method.
predictions = linearModel.transform(test_data)
#see prediction schema
predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [111]:
predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[0.0,0.0,0.0,0.0,...|[2.97743304250235...|[0.95154415140648...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[3.17336551678498...|[0.95981957999149...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[3.07907523294043...|[0.95602131957602...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[1.82751900796753...|[0.86146590472789...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[1.51694883396958...|[0.82008874138397...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [113]:
#model evaluations
cm = predictions.select("label", "prediction")
cm.groupby('label').agg({'label' : 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        7455|
|  1.0|        2328|
+-----+------------+



In [114]:
cm.groupby('prediction').agg({'prediction': 'count'}).show()

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|       0.0|             7771|
|       1.0|             2012|
+----------+-----------------+



In [115]:
#accuracy
cm.filter(cm.label == cm.prediction).count() / cm.count()

0.9676990698149852

In [116]:
### Use ROC 
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print(evaluator.evaluate(predictions))
print(evaluator.getMetricName())

0.9995083905494825
areaUnderROC


In [117]:
#hyperparameter tuning
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5])
             .build())

In [118]:
from time import *
start_time = time()

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(train_data)
# likely take a fair amount of time
end_time = time()
elapsed_time = end_time - start_time
print("Time to train model: %.3f seconds" % elapsed_time)

Time to train model: 453.401 seconds


In [119]:
#cv model evaluator
def accuracy_m(model): 
    predictions = model.transform(test_data)
    cm = predictions.select("label", "prediction")
    acc = cm.filter(cm.label == cm.prediction).count() / cm.count()
    print("Model accuracy: %.3f%%" % (acc * 100)) 
print(accuracy_m(model = cvModel))


Model accuracy: 100.000%
None


In [120]:
bestModel = cvModel.bestModel
bestModel.extractParamMap()

{Param(parent='LogisticRegression_a204be12eb61', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_a204be12eb61', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_a204be12eb61', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto',
 Param(parent='LogisticRegression_a204be12eb61', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LogisticRegression_a204be12eb61', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_a204be12eb61', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LogisticRegression_a204be12eb61', name='maxBlockSizeInMB', doc='maximum memory in MB for s